In [ ]:
from Database import ChromaDB
chroma = ChromaDB()
vector_store = chroma.get_vector_store(collection_name='test_collection')
chroma.current_collection

In [ ]:
import json
with open("./競賽資料集/dataset/preliminary/questions_example.json", "r") as q:
    question_set = json.load(q)

question_set

{'questions': [{'qid': 1,
   'source': [442, 115, 440, 196, 431, 392, 14, 51],
   'query': '匯款銀行及中間行所收取之相關費用由誰負擔?',
   'category': 'insurance'},
  {'qid': 2,
   'source': [475, 325, 578, 428, 606, 258, 275, 565],
   'query': '本公司應在效力停止日前多少天以書面通知要保人？',
   'category': 'insurance'},
  {'qid': 3,
   'source': [83, 476, 179, 148],
   'query': '「完全失能保險金」的受益人是否可以指定或變更?',
   'category': 'insurance'},
  {'qid': 4,
   'source': [186, 627, 536, 179, 174, 178],
   'query': '本契約內容的變更應經由誰同意並批註？',
   'category': 'insurance'},
  {'qid': 5,
   'source': [179, 631, 279, 162],
   'query': '「基本保額」是指什麼？',
   'category': 'insurance'},
  {'qid': 6,
   'source': [275, 242, 338, 116],
   'query': '要保人或受益人應於知悉本公司應負保險責任之事故後幾日內通知本公司？',
   'category': 'insurance'},
  {'qid': 7,
   'source': [486, 578, 439, 107],
   'query': '身故保險金的給付會使本契約效力如何變化? ',
   'category': 'insurance'},
  {'qid': 8,
   'source': [428, 578, 253, 244, 188, 353, 78],
   'query': '要保人在訂立本契約時，是否應對於本公司要保書書面詢問的告知事項據實說明？',
   'category': 'insurance

In [ ]:
# example

Q = question_set['questions'][20]
# {'qid': 1,
#  'source': [442, 115, 440, 196, 431, 392, 14, 51],
#  'query': '匯款銀行及中間行所收取之相關費用由誰負擔?',
#  'category': 'insurance'}


filter_conditions = {
    "$and": [
        {"category": {"$eq": f"{Q['category']}"}},
        {"source": {"$in": [str(i) for i in Q['source']]}}
    ]
}
vector_store.similarity_search(Q['query'],filter=filter_conditions,k=1)[0].metadata


{'category': 'insurance',
 'file_source': 'corpus_insurance.json',
 'page_number': 1,
 'source': '392',
 'source_id': '392'}

In [ ]:
answers = []

for Q in question_set['questions']:
    filter_conditions = {
        "$and": [
            {"category": {"$eq": f"{Q['category']}"}},
            {"source": {"$in": [str(i) for i in Q['source']]}}
        ]
    }
    chroma_output = vector_store.similarity_search(Q['query'],filter=filter_conditions,k=1)

    if chroma_output:
        query_result = chroma_output[0].metadata
    else:
        print(f"Find no answer for {Q['qid']}")
        query_result = {'category': 'not found',
                        'file_source': 'not found',
                        'page_number': -1,
                        'source': '-1',
                        'source_id': '-1'}

    answers.append({
        'qid': Q['qid'],
        'retrieve': int(query_result['source_id']),
        'category': query_result['category']
    })

with open("./predict_answers_1011.json", "w") as Output:
    json.dump({"answers": answers},Output,ensure_ascii=False,indent=4)
        

Find no answer for 70
Find no answer for 72


In [ ]:
## then use calculateScore.py to calculate Precision@1
## calculateScore.py --ground_truths_path ground_truths_example.json --predictions_path predict_answers_1011.json